### Algorithm Soft Actor-Critic<br>
FROM PAPER
***
**Input:**  $\theta_1, \theta_2, \phi .$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$\bar{\theta}_1\leftarrow \theta_1 ,\bar{\theta}_2\leftarrow \theta_2$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$D \leftarrow \varnothing$<br>
&nbsp;&nbsp;&nbsp;&nbsp;**for** each iteration **do**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**for** each environment step **do**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$a_t \sim \pi_\phi(a_t|s_t)$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$s_{t+1}\sim p(s_{t+1}|s_t, a_t)$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$D \leftarrow D\cup\{(s_t,a_t,r(s_t,a_t),s_{t+1})\}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**end for**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**for** each gradient step **do**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\theta_i\leftarrow\theta_i - \lambda_Q\hat{\nabla}_{\theta_i}J_Q(\theta_i) $&nbsp;&nbsp;for $i\in \{1,2\}$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Update the Q-function parameters<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\phi \leftarrow \phi - \lambda_\pi\hat{\nabla}_\phi J_\pi(\phi)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Update policy weights<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\psi\leftarrow \lambda_V\hat{\nabla}_\psi J_V(\psi)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Adjust temperature<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\bar{\theta}_i\leftarrow \tau\theta_i+(1-\tau)\bar{\theta}_i $&nbsp; for$i\in \{1,2\}$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Update target network weights<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**end for**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**end for**<br>
**Output:**&nbsp;$\theta_1,\theta_2,\phi$

### Main Formula:
1. Soft Bellman residual:
$$
J_Q(\theta)=\mathbb{E}_{(s_t,a_t)\sim D}\big[\frac{1}{2}(Q_\theta(s_t,a_t)-{\cal{T}}^\pi Q(s_t,a_t))^2\big]\tag{1}
$$
Soft Q-value function:
$$
{\cal{T}}^\pi Q(s_t,a_t) \triangleq r(s_t,a_t)+\gamma\mathbb{E}_{s_{t+1}\sim p}[V_\bar{\theta}(s_{t+1})]\tag{2}
$$
Soft state value function:
$$
V(s_t) = \mathbb{E}_{a_t\sim \pi}[Q(s_t,a_t)-\alpha\log\pi(a_t|s_t)]\tag{3}
$$
由此推出Soft Bellman residual导数:
$$
\hat{\nabla}_\theta J_Q(\theta)=\nabla_\theta Q_\theta(a_t,s_t)(Q_\theta(s_t,a_t)-(r(s_t,a_t)+\gamma(Q_{\bar{\theta}}(s_{t+1},a_{t+1})-\alpha\log(\pi_\phi(a_{t+1}|s_{t+1}))))\tag{4}
$$
2. Policy Loss:
$$
J_\pi(\phi)=-\mathbb{E}_{s_t\sim D}\big[\mathbb{E}_{a_t\sim \pi_\phi}[Q_\phi(s_t,a_t)-\alpha\log(\pi_\phi(a_t|s_t))]\big]\tag{5}
$$
又
$$
a_t=f_\phi(\epsilon_t;s_t),\tag{6}
$$
所以可写成:
$$
J_\pi(\phi)=-\mathbb{E}_{s_t\sim D,\;\epsilon_t\sim N}[Q_\theta(s_t,f_\phi(\epsilon_t;s_t))-\alpha\log\pi_\phi(f_\phi(\epsilon_t;s_t)|s_t)]\tag{7}
$$
所以其导数形式为:
$$
\hat{\nabla}_\phi J_\pi(\phi)=\nabla_\phi\alpha\log(\pi_\phi(a_t|s_t))+\big(\nabla_{a_t}\alpha\log(\pi_\phi(a_t|s_t))-\nabla_{a_t}Q(s_t,a_t)\big)\nabla_\phi f_\phi(\epsilon_t;s_t),\tag{8}
$$
3. 自适应temperature $\alpha$(论文中说$\alpha$和$Q$、$\pi$是对偶问题，有点不明白):
$$
\alpha^*_t=\arg {\min_{\alpha_t}}\mathbb{E}_{a_t\sim\pi^*_t}[-\alpha_t\log\pi^*_t(a_t|s_t;a_t)-\alpha_t\bar{H}]\tag{9}
$$


### Formula Proofs
1. Proof formula $f_2(f_3)$:<br>
从最开始的动作转移开始，对于在每次$state$采取的$action$所获得的$soft\ reward$都可定义如下:<br><br>
$$
r_{soft}(s_t,a_t)=r(s_t,a_t)+\gamma\alpha\mathbb{E}_{s_{t+1}\sim \rho}H(\pi(\cdot|s_{t+1}))\tag{10}
$$<br>
将其带入到原始的$Q\ function\ : Q(s_t,a_t)=r(s_t,a_t)+\gamma\mathbb{E}_{s_{t+1},a_{t+1}}[Q(s_{t+1},a_{t+1})]$中得:<br><br>
$$
\begin{aligned}
Q_{soft}(s_t,a_t)&=r(s_t,a_t)+\gamma\alpha\mathbb{E}_{s_{t+1}\sim\rho}H(\pi(\cdot|s_{t+1}))+\gamma\mathbb{E}_{s_{t+1},a_{t+1}}[Q_{soft}(s_{t+1},a_{t+1})]\\
&=r(s_t,a_t)+\gamma\mathbb{E}_{s_{t+1}\sim\rho,a_{t+1}\sim\pi}[Q_{soft}(s_{t+1},a_{t+1})]+\gamma\alpha\mathbb{E}_{s_{t+1}\sim\rho}H(\pi(\cdot|s_{t+1}))\\
&=r(s_t,a_t)+\gamma\mathbb{E}_{s_{t+1}\sim\rho,a_{t+1}\sim\pi}[Q_{soft}(s_{t+1},a_{t+1})]+\gamma\mathbb{E}_{s_{t+1}\sim\rho}\mathbb{E}_{a_{t+1}\sim\pi}[-\alpha\log\pi(a_{t+1}|s_{t+1})]\\
&=r(s_t,a_t)+\gamma\mathbb{E}_{s_{t+1}\sim\rho}[\mathbb{E}_{a_{t+1}\sim\pi}[Q_{soft}(s_{t+1},a_{t+1})-\alpha\log(\pi(a_{t+1}|s_{t+1}))]]
\end{aligned}\tag{11}
$$
2. Proff formula $f$

### [相对熵(KL散度)](https://blog.csdn.net/tsyccnh/article/details/79163834)
对于同一个随机变量x单独的概率分布P(x)和Q(x)，用来衡量其分布的差异
$$
D_{KL}(p||q)=\sum_{i=1}^n p(x_i)\log[\frac{p(x_i)}{q(x_i)}]
$$
$D_{KL}$越接近于0，$p,q$分布越接近<br><br>
展开得
$$
\begin{aligned}
D_{KL}(p||q)&=\sum_{i=1}^np(x_i)\log(p(x_i))-\sum^n_{i+1}p(x_i)\log(q(x_i)) \\
&=\underbrace{-H(p(x))}_{熵}+\underbrace{[-\sum^n_{i=1}p(x_i)\log(q(x_i))]}_{交叉熵}
\end{aligned}
$$
在分类问题中，假设label为p，则前部分是不变的，即只需计算后部分，即**交叉熵**

### Landscape
讲了那么多高深的理论，其实就是在actor和critic计算loss的时候分别考虑了entorpy，增加探索。也就是说critic既然是知道actor前进的，那么你也必须将增加探索这四个字铭记在心，不然你怎么知道actor的动作呢<br>
其次，deterministic版本的sac是没有entorpy的，因为是确定性策略所以没有entropy的来源(所以确定性策略的sac其实比TD3效果可能还要差)<br>
要有entorpy必须是normal distribution，所以其实要改成离散的动作空间，也就几行代码的事

### 代码实现的Tips
1. 使用torch.no_grad(), 而不是.detach()更加直观

### 相较于SAC的五大EDIT

1. 将Critic network 改为input state, output Q_value
2. 将Actor network 改为softmax输出
3. 将Critic更新计算Q改为根据Actor进行gather
4. alpha更新通过Actor进行gather
5. Actor更新乘上了每个动作的概率，以此来达到方差最小化

---Atari环境对学习率非常敏感，1e-3和1e-4差别非常大---<br>
加上nn.BatchNorm2d(32)并不能加快学习

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import gym
import random
import numpy as np
from itertools import count
import matplotlib.pyplot as plt
from collections import namedtuple, deque
import time
import os
import sys
sys.path.append('../')
from utils.wrappers import make_atari, wrap_deepmind, wrap_pytorch
%matplotlib inline
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class ReplayBuffer:
    def __init__(self, state_dims, buffer_size, batch_size):
        self.state = np.zeros((buffer_size, state_dims[0], state_dims[1], state_dims[2]), dtype=np.float32)
        self.action = np.zeros(buffer_size, dtype=np.float32)
        self.next_state = np.zeros((buffer_size, state_dims[0], state_dims[1], state_dims[2]), dtype=np.float32)
        self.reward = np.zeros(buffer_size, dtype=np.float32)
        self.done = np.zeros(buffer_size, dtype=np.float32)
        self.batch_size = batch_size
        self.buffer_size = buffer_size
        self.size, self.current_index = 0, 0
    
    def store(self, state, action, next_state, reward, done):
        self.state[self.current_index] = state
        self.action[self.current_index] = action
        self.next_state[self.current_index] = next_state
        self.reward[self.current_index] = reward
        self.done[self.current_index] = done
        self.current_index = (self.current_index + 1) % self.buffer_size
        self.size = min((self.size + 1), self.buffer_size)
    
    def sample(self):
        idx = np.random.choice(self.size, self.batch_size)
        return dict(state = torch.FloatTensor(self.state[idx]).to(device),
                    action = torch.LongTensor(self.action[idx]).unsqueeze(1).to(device),
                    next_state = torch.FloatTensor(self.next_state[idx]).to(device),
                    reward = torch.FloatTensor(self.reward[idx]).unsqueeze(1).to(device),
                    done = torch.FloatTensor(self.done[idx]).unsqueeze(1).to(device))
    
    def __len__(self):
        return self.size

In [3]:
def weights_init_(m):
    if isinstance(m, nn.Linear):
        # 使用 std = $$\text{std} = \text{gain} \times \sqrt{\frac{2}{\text{fan\_in} + \text{fan\_out}}}$$
        # 来代替高斯分布(0, std ^ 2)的std
        torch.nn.init.xavier_uniform_(m.weight, gain=1)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)

#         self.common_layer = nn.Sequential(
#             nn.Conv2d(state_dims[0], 32, kernel_size=5, stride=1, padding=2),
#             nn.MaxPool2d(2),
#             nn.ReLU(),
#             nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=1),
#             nn.MaxPool2d(2),
#             nn.ReLU(),
#             nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=1),
#             nn.MaxPool2d(2),
#             nn.ReLU(),
#             nn.Conv2d(64, 64, kernel_size=3, stride=1),
#             nn.MaxPool2d(2),
#             nn.ReLU()
#         )

# EDIT 1
class Critic(nn.Module):
    def __init__(self, state_dims, action_dim, hidden_dim=512):
        super(Critic, self).__init__()
        
        self.common_layer = nn.Sequential(
            nn.Conv2d(state_dims[0], 32, kernel_size=8, stride=4),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.linear1 = nn.Linear(7 * 7 * 64, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, action_dim)
        
        self.apply(weights_init_)
        
    def forward(self, state):
        common = self.common_layer(state)
        common = common.view(common.size(0), -1)
        
        linear = F.relu(self.linear1(common))
        value = self.linear2(linear)
        return value

# EDIT 2
class Actor(nn.Module):
    def __init__(self, state_dims, action_dim, hidden_dim=512):
        super(Actor, self).__init__()
        
        self.common_layer = nn.Sequential(
            nn.Conv2d(state_dims[0], 32, kernel_size=8, stride=4),\
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.linear1 = nn.Linear(7 * 7 * 64, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, action_dim)
        
        self.apply(weights_init_)
        
    def forward(self, state):
        common = self.common_layer(state)
        common = common.view(common.size(0), -1)
        
        linear = F.relu(self.linear1(common))
        action_probs = F.softmax(self.linear2(linear), dim=1)
        max_action_prob = torch.argmax(action_probs, dim=1)
        
        dist = Categorical(action_probs)
        
        action = dist.sample()
        action_log_prob = dist.logits
        
        return action.unsqueeze(1), action_probs, action_log_prob

In [4]:
def take_action(state, timesteps):
    if start_steps > timesteps:
        action = env.action_space.sample()
    else:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        with torch.no_grad():
            action, _, _ = actor(state)
        action = action.item()
    return action

In [5]:
## hyperparameters

env_name = "PongNoFrameskip-v4"
start_steps = 10000

# env_name = "Breakout-v0"
# start_steps = 500

env = make_atari(env_name)
env = wrap_deepmind(env)
env = wrap_pytorch(env)

algorithm_id = "soft_actor_critic_discrete_image"

buffer_size = int(1e6)
batch_size = 64
episodes = 10000

learning_rate = 1e-5
gamma = 0.99
soft_tau = 5e-3
actor_update = 2
automatic_entropy_tuning = True

## hyperparameters

current_time = time.strftime('%Y-%m-%d_%H:%M:%S',time.localtime(time.time()))
ROOT_DIR = "../running_log/{}/{}/{}".format(algorithm_id, env_name, current_time)
model_dir = os.path.join(ROOT_DIR, "model")
plot_dir = os.path.join(ROOT_DIR, "tensorboard")
os.makedirs(model_dir)
os.makedirs(plot_dir)
writer = SummaryWriter(plot_dir, comment="learning_rate={}-batch_size={}-start_steps={}"
                       .format(learning_rate , batch_size, start_steps))

# env = gym.make(env_name)
# state_dim = env.observation_space.shape[0]
state_dims = env.observation_space.shape
action_dim = env.action_space.n

critic_1 = Critic(state_dims, action_dim).to(device)
critic_2 = Critic(state_dims, action_dim).to(device)
target_critic_1 = Critic(state_dims, action_dim).to(device)
target_critic_2 = Critic(state_dims, action_dim).to(device)
actor = Actor(state_dims, action_dim).to(device)

target_critic_1.load_state_dict(critic_1.state_dict())
target_critic_2.load_state_dict(critic_2.state_dict())

critic_optimizer_1 = optim.Adam(critic_1.parameters(), lr=learning_rate)
critic_optimizer_2 = optim.Adam(critic_2.parameters(), lr=learning_rate)
actor_optimizer = optim.Adam(actor.parameters(), lr=learning_rate)

buffer = ReplayBuffer(state_dims, buffer_size, batch_size)

# torch.prod()
# Returns the product of all elements in the :attr:`input` tensor
# 返回输入张量中所有元素的乘积(可指定维度)
if automatic_entropy_tuning:
#     target_entropy = - torch.prod(torch.Tensor(env.action_space.shape).to(device)).item() # -4.0
    target_entropy = - 1.0
    log_alpha = torch.zeros(1, requires_grad=True, device=device) # tensor([0.], device='cuda:0', requires_grad=True)
    alpha = log_alpha.exp()
    alpha_optim = optim.Adam([log_alpha], lr=learning_rate)

In [6]:
def sac_train(updates, steps_):
    global alpha
    
    for i in range(steps_):
        samples = buffer.sample()
        state, action, next_state = samples["state"], samples["action"], samples["next_state"]
        reward, done = samples["reward"], samples["done"]

        # update critic
        with torch.no_grad():
            # EDIT 3
            next_action, _, next_action_log_probs = actor(next_state)
            next_action_log_probs = next_action_log_probs.gather(1, next_action.long())
            target_Q_1 = target_critic_1(next_state).gather(1, next_action.long())
            target_Q_2 = target_critic_2(next_state).gather(1, next_action.long())
            Q_target_next = torch.min(target_Q_1, target_Q_2) - alpha * next_action_log_probs
            next_q_value = reward + (1.0 - done) * gamma * Q_target_next

        Q_1 = critic_1(state).gather(1, action)
        Q_2 = critic_2(state).gather(1, action)
        critic_loss_1 = F.mse_loss(next_q_value, Q_1)
        critic_loss_2 = F.mse_loss(next_q_value, Q_2)

        critic_optimizer_1.zero_grad()
        critic_loss_1.backward()
        critic_optimizer_1.step()

        critic_optimizer_2.zero_grad()
        critic_loss_2.backward()
        critic_optimizer_2.step()

        # update actor
        # EDIT 5
        if i % actor_update == 0:
            actions, action_probs, action_log_probs = actor(state)
            min_Q_value = torch.min(critic_1(state), critic_2(state))
            actor_loss = (alpha * action_log_probs - min_Q_value) * action_probs
            actor_loss = torch.sum(actor_loss, dim=1, keepdim=True).mean()

            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()

            # update entropy_tuning

            if automatic_entropy_tuning:
                # EDIT 4
                action_log_probs = action_log_probs.gather(1, actions.long())
                alpha_loss = - log_alpha * (action_log_probs.detach() + target_entropy)
                alpha_loss = alpha_loss.mean()

                alpha_optim.zero_grad()
                alpha_loss.backward()
                alpha_optim.step()

                alpha = log_alpha.exp()

            # update parameter
            for target_param, param in zip(target_critic_1.parameters(), critic_1.parameters()):
                target_param.data.copy_(target_param.data*(1.0-soft_tau) + param.data * soft_tau)
            for target_param, param in zip(target_critic_2.parameters(), critic_2.parameters()):
                target_param.data.copy_(target_param.data*(1.0-soft_tau) + param.data * soft_tau)

    writer.add_scalars("Loss/Critic", {"critic_1":critic_loss_1,
                                "critic_2":critic_loss_2}, updates)
    writer.add_scalar("Loss/Actor", actor_loss, updates)
    writer.add_scalar("Loss/Alpha", alpha_loss, updates)

In [ ]:
updates, timesteps, done_time = 0, 0, 0

for episode in range(episodes):
    state = env.reset()
    episode_reward = 0
    for i in count():
        timesteps += 1
        action = take_action(state, timesteps)
        
        next_state, reward, done, _ = env.step(action)
        
        buffer.store(state, action, next_state, reward, done)
        
        state = next_state
        episode_reward += reward
        
        if done:
            if len(buffer) > batch_size:
                sac_train(updates, i+1)
                updates += 1
            writer.add_scalar("Episode_step", i, done_time)
            done_time += 1
            break
        
    writer.add_scalar("Reward", episode_reward, episode)
torch.save(actor, model_dir + "/actor_model.pth")

## Reference:
[SOFT ACTOR-CRITIC FOR DISCRETE ACTION SETTINGS](https://arxiv.org/pdf/1910.07207.pdf)

[Deep-Reinforcement-Learning-Algorithms-with-PyTorch/agents/actor_critic_agents/SAC_Discrete.py](https://github.com/p-christ/Deep-Reinforcement-Learning-Algorithms-with-PyTorch/blob/master/agents/actor_critic_agents/SAC_Discrete.py)

[https://github.com/Jabberwockyll/deep_rl_ale/blob/master/experience_memory.py](https://github.com/Jabberwockyll/deep_rl_ale/blob/master/experience_memory.py)